In [17]:
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir, path

In [4]:
DATASET = "dataset"
TRAINSET = "{0}/keras_png_slices_train".format(DATASET)
TESTSET = "{0}/keras_png_slices_test".format(DATASET)

In [23]:
def train_set():
    trainDataset = []
    for i in listdir(TRAINSET):
        source = "{}/{}".format(TRAINSET, i)
        trainDataset.append(
            mpimg.imread(source)
        )
    return trainDataset

def test_set():
    testDataset = []
    for i in listdir(TESTSET):
        source = "{}/{}".format(TESTSET, i)
        testDataset.append(
            mpimg.imread(source)
        )
    return testDataset

In [19]:
trainDataset = train_set()

In [24]:
testDataset = test_set()